In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.executable

'/home/rio/miniconda3/envs/multibind/bin/python'

In [3]:
import numpy as np
import pandas
import multibind

In [4]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '../annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

Using device: cuda:0


In [5]:
data = mb.bindome.datasets.SELEX.get_data()

cardiac_complexes
# filenames 22
(22, 2)
PRJEB14744
# filenames 13
(13, 2)
PRJEB20112
# filenames 46
(46, 2)
PRJEB3289
# filenames 73
(73, 2)
PRJEB9797
# filenames 184
(184, 2)


In [6]:
# data_sel = data[data['filename'].str.contains('GATA') | data['filename'].str.contains('FOXA1')]
# read_tf = mb.bindome.datasets.SELEX.load_read_counts('GATA4', data)
# read_tf = mb.bindome.datasets.SELEX.load_read_counts(log_each=50, n_sample=100) #  stop_at=50)
# df = mb.bindome.datasets.ProBound.GR_mult_conc()
# df = df.sort_values(1, ascending=False).reset_index(drop=True)

In [7]:
# data[data['accession'] == 'cardiac_complexes']

In [8]:
data['tf.name'].value_counts()

ZEROCYCLE      73
ELK1           38
ETV4           28
ELK3           28
ELF2           24
EGR1           23
ELF3           20
ELF4           16
FOXA1          16
ALX4           12
ETV5           12
ETS1           10
ALX1            8
CTCF            4
ETV6            4
GATA4           3
TBX5            3
R46             3
NKX2-5          3
NKX2-5TBX5      3
NKX2-5GATA4     3
GATA4TBX5       3
Name: tf.name, dtype: int64

In [13]:
import scanpy as sc
import anndata
import os
import gc


In [34]:
tf_names = set(data['tf.name'].value_counts().index)
tf_names = {s for s in tf_names if not s in {'R46', 'GATA4', 'GATA4TBX5', 'NKX2-5', 'NKX2-5TBX5', 'NKX2-5GATA4', 'TBX5', 'ZEROCYCLE'}}
tf_names

{'ALX1',
 'ALX4',
 'CTCF',
 'EGR1',
 'ELF2',
 'ELF3',
 'ELF4',
 'ELK1',
 'ELK3',
 'ETS1',
 'ETV4',
 'ETV5',
 'ETV6',
 'FOXA1'}

In [36]:
# loading with the fastest command takes about 5 minutes
for tf_name in tf_names:
    k = 11
    print(tf_name)
    data_sel_tf = data[(data['tf.name'] == tf_name)] #  & (grp['cycle'].astype(str) == '1')]
    data_sel_zero = data[(data['cycle'] == 0) & data['library'].isin(set(data[data['tf.name'] == tf_name]['library']))]  # & grp['accession'].isin(set(grp[grp['tf.name'] == tf]['accession']))]

    data_sel = pd.concat([data_sel_tf, data_sel_zero]).reset_index(drop=True)
    data_sel.shape
    
    # data_sel = data[data['accession'] != 'cardiac_complexes'] # .head(4)
    # data_sel = data_sel.head(5)

    # data_sel = data[data['tf.name'].str.contains('ELK1')].reset_index(drop=True)
    data_sel['cycle'] = data_sel['cycle'].astype(int)
    # data_sel['cycle']
    # print(data_sel_tf.shape[0], data_sel_zero.shape[0])
    # print('loading', tf, ':', library)
    # reads_tf = mb.bindome.datasets.SELEX.load_read_counts(tf, data=data_sel_tf)
    # reads_zero = mb.bindome.datasets.SELEX.load_read_counts(data=data_sel_zero, library=library)

    adata_kmers_bkp = '../../data/kmers_adata_%s_k%i.h5ad' % (tf_name, k)
    print(adata_kmers_bkp)
    if not os.path.exists(adata_kmers_bkp):
        kmers_by_k = {}
        for ri, r in data_sel.iterrows():
            if ri % 5 == 0:
                print('%i out of %i' % (ri, data_sel.shape[0]))
            key_added = r['filename'].replace('.fastq.gz', '').replace('.txt.gz', '')    

            print(r['path'])
            df_kmers = None
            if mb.bindome.datasets.SELEX.is_fastq(r['path']) and k >= 10:
                df_kmers = mb.tl.fastq2kmers(r['path'], k, log=True) # 
                df_kmers[df_kmers.columns[0]] = pd.arrays.SparseArray(df_kmers[df_kmers.columns[0]])
            else:
                read_tf = mb.bindome.datasets.SELEX.load_read_counts(data=data_sel[data_sel.index == ri], log_each=50, n_sample=100)
                next_df = read_tf[list(read_tf.keys())[0]]
                df_kmers = mb.tl.seqs2kmers(next_df['seq'], k, counts=next_df['counts'])

            df_kmers.columns = [key_added]

            # convert to sparse
            df_kmers[df_kmers.columns[0]] = pd.arrays.SparseArray(df_kmers[df_kmers.columns[0]])
            kmers_by_k[key_added] = df_kmers

            if len(kmers_by_k.keys()) % 25 == 0:
                print(len(kmers_by_k.keys()), 'out of', data_sel.shape[0])

            import warnings
            warnings.filterwarnings("ignore")

        from functools import reduce
        print('merging...')
        df_merged = reduce(lambda  left,right: pd.merge(left, right, left_index=True, right_index=True,
                                                    how='outer'), kmers_by_k.values()).fillna(0)
        print('merging done...')

        del kmers_by_k
        gc.collect()

        print('transposing...')
        df_merged = df_merged.T

        # df_merged

        print('convert to anndata...')
        adata = anndata.AnnData(df_merged)

        del df_merged
        gc.collect()

        data_sel2 = data_sel.copy()
        del data_sel
        gc.collect()

        data_sel2.index = adata.obs_names

        adata.obs = data_sel2
        adata.var_names = list(adata.var_names)

        print('writing')
        adata.write(adata_kmers_bkp, compression='lzf')
        print('writing done...')

    print('reading adata')
    adata = sc.read_h5ad(adata_kmers_bkp)
    print('done...')

ELF4
../../data/kmers_adata_ELF4_k11.h5ad
0 out of 20
../annotations/selex/PRJEB9797/ELF4_eDBD_1_KAN_TGCGGT40NTCA.fastq.gz
kmc -cs1553592 -k11 -ci1 -m8 ../annotations/selex/PRJEB9797/ELF4_eDBD_1_KAN_TGCGGT40NTCA.fastq.gz NA kmc_tmp 1> in.txt 2> err.txt
kmc_tools transform NA dump NA.txt


in1: 100% 


../annotations/selex/PRJEB9797/ELF4_eDBD_1_KAN_TGTTAA40NACT.fastq.gz
kmc -cs2136556 -k11 -ci1 -m8 ../annotations/selex/PRJEB9797/ELF4_eDBD_1_KAN_TGTTAA40NACT.fastq.gz NA kmc_tmp 1> in.txt 2> err.txt
kmc_tools transform NA dump NA.txt


in1: 100% 


../annotations/selex/PRJEB9797/ELF4_eDBD_1_KS_TCAGCA40NCTC.fastq.gz
kmc -cs1071816 -k11 -ci1 -m8 ../annotations/selex/PRJEB9797/ELF4_eDBD_1_KS_TCAGCA40NCTC.fastq.gz NA kmc_tmp 1> in.txt 2> err.txt
kmc_tools transform NA dump NA.txt


...

../annotations/selex/PRJEB20112/ZeroCycle_TGCGTA40NTCA_0_0.fastq.gz
kmc -cs897372 -k11 -ci1 -m8 ../annotations/selex/PRJEB20112/ZeroCycle_TGCGTA40NTCA_0_0.fastq.gz NA kmc_tmp 1> in.txt 2> err.txt
kmc_tools transform NA dump NA.txt


in1: 100% 


15 out of 16
../annotations/selex/PRJEB3289/ZeroCycle_TGTGTC20NGA_0_0.fastq.gz
kmc -cs41424 -k11 -ci1 -m8 ../annotations/selex/PRJEB3289/ZeroCycle_TGTGTC20NGA_0_0.fastq.gz NA kmc_tmp 1> in.txt 2> err.txt
kmc_tools transform NA dump NA.txt
merging...


in1: 100% 


merging done...
transposing...
convert to anndata...
writing
writing done...
reading adata
done...


In [37]:
print('here...')

here...


In [15]:
adata.to_df().T.sum(axis=0)

CTCF_full_AJ_TAGCGA20NGCT_1     1500.0
CTCF_full_AJ_TAGCGA20NGCT_2     1500.0
CTCF_full_AJ_TAGCGA20NGCT_3     1500.0
CTCF_full_AJ_TAGCGA20NGCT_4     1500.0
ZeroCycle_ES0_TAGCGA20NGCT_0    1500.0
ZeroCycle_TAGCGA20NGCT_0_0      1500.0
dtype: float32

In [16]:
!ls -ltrh $adata_kmers_bkp

-rwxrwxrwx 1 rio rio 160K Jun 20 18:39 ../../data/kmers_adata_CTCF_k6.h5ad


In [ ]:
size = 0
for p in data_sel['path']:
    size += os.path.getsize(p) / 1e6
print(size)

In [ ]:
# # %load_ext line_profiler
# %lprun -f get_kmers get_kmers(next_df['seq'], 6, counts=next_df['counts'])
# %time kmers = get_kmers(next_df['seq'], 6, counts=next_df['counts'])

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 9, 2

In [ ]:
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
sc.set_figure_params(facecolor='white')

In [ ]:
import seaborn as sns

In [ ]:
adata

In [ ]:
# adata.obs

In [ ]:
# adata.to_df()

In [ ]:
g = sns.clustermap(adata.to_df().T.corr(), cmap='Reds', figsize=[6, 6], xticklabels=False, yticklabels=False)

In [ ]:
import pandas as pd

In [ ]:
adata.obs

In [ ]:
# adata.obs_names = adata.obs['_index']
adata.obs['is.zero'] = pd.Categorical(adata.obs_names.str.contains('Zero'))

In [ ]:
adata.obs['tf.name'] = list(data_sel['tf.name'])
adata.obs['accession'] = list(data_sel['accession'])
adata.obs['cycle'] = adata.obs['cycle'].astype('category')

In [ ]:
adata.obs['tf.name'].value_counts()

In [ ]:
for tf in set(adata.obs['tf.name']):
    print(tf, len(set(adata.obs[adata.obs['tf.name'] == tf]['accession'])))

In [ ]:
adata[adata.obs['tf.name'] == 'ELK1',:].obs

In [ ]:
g = sns.clustermap(adata[adata.obs['tf.name'] == tf_name,:].to_df().T.corr(), cmap='Reds', figsize=[6, 6], xticklabels=False, yticklabels=False)

In [ ]:
sc.pl.umap(adata, color=['tf.name', 'accession', 'library', 'cycle'], s=100)
sc.pl.pca(adata, color=['tf.name', 'accession', 'library', 'cycle'], s=100)


In [ ]:
adata.to_df()

In [ ]:
adata.obs

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.executable

import numpy as np
import pandas
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '/mnt/c/Users/ignacio.ibarra/Dropbox/annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

In [ ]:
df = mb.bindome.datasets.ProBound.GR_mult_conc()
df = df.sort_values(1, ascending=False).reset_index(drop=True)

In [ ]:
data = df.head(1000)

In [ ]:
n_rounds = 1

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 9, 2

In [ ]:
# assign batch and data type

data['batch'] = 1
data['is_count_data'] = 1

print('loading object (# entries)', data.shape[0])
dataset = mb.datasets.SelexDataset(data, n_rounds=n_rounds, single_encoding_step=True)
train = tdata.DataLoader(dataset=dataset, batch_size=256, shuffle=True)

model_by_k_next, res_next = mb.tl.train_iterative(train, device, min_w=16, max_w=18, n_kernels=2)                
res = []
model_by_k = {}


In [ ]:
adata.obs['is.zero'] = adata.obs_names.str.contains('Zero').astype('categorical')

In [ ]:
adata

In [ ]:
sc.pl.umap(adata, color='is.zero', s=50)

In [ ]:
sc.pl.umap(adata)